In [ ]:
#r "nuget:Microsoft.Spark"

Installed Packages Microsoft.Spark, 2.1.0

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [ ]:
using Microsoft.Spark;
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

// run spark : spark-submit --class org.apache.spark.deploy.dotnet.DotnetRunner --master local D:\3bStudio\Sandbox\spark-program\FirstSparkProgram\bin\Debug\net6.0\microsoft-spark-3-0_2.12-2.1.0.jar debug
var spark = SparkSession
    .Builder()
    .AppName("testing interactive app")
    .GetOrCreate();

[2022-03-11T10:03:08.7845521Z] [SURFACE-WHA] [Info] [ConfigurationService] 'DOTNETBACKEND_PORT' environment variable is not set.
[2022-03-11T10:03:08.7867738Z] [SURFACE-WHA] [Info] [ConfigurationService] Using port 5567 for connection.
[2022-03-11T10:03:08.7874468Z] [SURFACE-WHA] [Info] [JvmBridge] JvMBridge port is 5567
[2022-03-11T10:03:08.7880957Z] [SURFACE-WHA] [Info] [JvmBridge] The number of JVM backend thread is set to 10. The max number of concurrent sockets in JvmBridge is set to 7.


In [ ]:
// Chargement des tables de données
var products_table = spark.Read().Parquet("D:/3bStudio/Sandbox/spark-program/resources/products_parquet");
var sales_table = spark.Read().Parquet("D:/3bStudio/Sandbox/spark-program/resources/sales_parquet");
var sellers_table = spark.Read().Parquet("D:/3bStudio/Sandbox/spark-program/resources/sellers_parquet");

In [ ]:
// Comptage du nombre d'éléments par table
var productCount = products_table.Count();
var salesCount = sales_table.Count();
var sellersCount = sellers_table.Count();

In [ ]:
// Affichage des résultats
Console.WriteLine($"______________nb of products: {productCount}");
Console.WriteLine($"______________nb of sales: {salesCount}");
Console.WriteLine($"______________nb of Sellers: {sellersCount}");

______________nb of products: 75000000
______________nb of sales: 20000040
______________nb of Sellers: 10


In [ ]:
// Get 10 first most sold disctinct products
sales_table.GroupBy("date")
            .Agg(Functions.CountDistinct("product_id"))
            .Alias("distinct_products_sold")
            .Show();

+----------+-----------------+
|      date|count(product_id)|
+----------+-----------------+
|2020-07-03|           100017|
|2020-07-07|            99756|
|2020-07-01|           100337|
|2020-07-08|            99662|
|2020-07-04|            99791|
|2020-07-10|            98973|
|2020-07-09|           100501|
|2020-07-06|           100765|
|2020-07-02|            99807|
|2020-07-05|            99796|
+----------+-----------------+



In [ ]:
// Uses Spark SQL to get top 10 most sold products
sales_table.CreateOrReplaceTempView("salesTable");
spark.Sql("SELECT date, count(*) as distinct_products_sold FROM salesTable GROUP BY date")
    .Show();

+----------+----------------------+
|      date|distinct_products_sold|
+----------+----------------------+
|2020-07-03|               2000651|
|2020-07-07|               2000807|
|2020-07-01|               2001370|
|2020-07-08|               2000451|
|2020-07-04|               2000949|
|2020-07-10|               1998505|
|2020-07-09|               2000402|
|2020-07-06|               2001302|
|2020-07-02|               1998985|
|2020-07-05|               1996618|
+----------+----------------------+

